# Question 1

In [1]:
# for basic file navigation and driving mount
import os
from google.colab import drive

# some basic libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# importing for basic image operations
from PIL import Image
from PIL import ImageOps
import seaborn as sns

# importing pretrained models
from tensorflow.keras.applications import efficientnet_v2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG19

# for splitting data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# prerequisites for training models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from sklearn.metrics import confusion_matrix
import seaborn as sns

# for selecting better metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
# define TPU strategy for effecient computation
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

Device: grpc://10.112.118.226:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.112.118.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.112.118.226:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# load data from cifar 10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# converting to categorical features
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

170508288/170498071 [==============================] - 2s 0us/step


In [9]:
img_size = 32

with strategy.scope():

  #using tensorflow API for pretrained models and intializing them  
  effecientnet_base = VGG19(input_shape=(img_size, img_size, 3), include_top = False, classes = 10, weights = 'imagenet')

  #creating Sequential models
  model = Sequential(
    [effecientnet_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = "relu", input_dim=512),
    tf.keras.layers.Dropout(0.005),
    tf.keras.layers.Dense(512, activation = "sigmoid"),
    tf.keras.layers.Dense(256, activation = "relu"),
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(10, activation = "softmax")])
  
  #compiling models with accuracy
  model.compile(tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), 'categorical_crossentropy', metrics=['accuracy'])
  
  #fit model
  model.fit(x_train, y_train, epochs = 10, batch_size = 100)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
500/500 [==============================] - 33s 25ms/step - loss: 1.2696 - accuracy: 0.5610
Epoch 2/10
500/500 [==============================] - 13s 26ms/step - loss: 0.6489 - accuracy: 0.7899
Epoch 3/10
500/500 [==============================] - 13s 25ms/step - loss: 0.4715 - accuracy: 0.8468
Epoch 4/10
500/500 [==============================] - 13s 25ms/step - loss: 0.3538 - accuracy: 0.8842
Epoch 5/10
500/500 [==============================] - 13s 26ms/step - loss: 0.2736 - accuracy: 0.9121
Epoch 6/10
500/500 [==============================] - 13s 26ms/step - loss: 0.2026 - accuracy: 0.9341
Epoch 7/10
500/500 [==============================] - 13s 26ms/step - loss: 0.1598 - accuracy: 0.9496
Epoch 8/10
500/500 [==============================] - 13s 26ms/step - loss: 0.1209 - accuracy: 0.9609
Epoch 9/10
500/500 [==============================] - 13s 26ms/step - loss: 0.1038 - accuracy: 0.9677
Epoch 10/10
500/500 [==============================] - 13s 25ms/step - loss: 0.088

In [10]:
# evaluate model performance on test set
with strategy.scope():
  model.evaluate(x_test, y_test)

313/313 [==============================] - 9s 20ms/step - loss: 0.6693 - accuracy: 0.8314


In [ ]:
# testing the data and getting predictions
with strategy.scope():
  predictions = model.predict(x_test)

In [ ]:
# plotting the heatmap
plt.figure(figsize = (20, 10))
sns.heatmap(confusion_matrix(y_test.argmax(axis = 1), predictions.argmax(axis = 1)), annot=True, fmt='.2f')
plt.show()

# Question 2


In [11]:
# for basic file navigation and driving mount
import os
from google.colab import drive

# some basic libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# importing for basic image operations
from PIL import Image
from PIL import ImageOps

# importing pretrained models
from tensorflow.keras.applications import efficientnet_v2
from tensorflow.keras.datasets import mnist


# for splitting data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# prerequisites for training models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from sklearn.svm import SVC

# for selecting better metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [12]:
# define TPU strategy for effecient computation
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

Device: grpc://10.112.118.226:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.112.118.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.112.118.226:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [13]:
# load data from cifar 10 dataset from default train test split (around 85:15)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# convert to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

11501568/11490434 [==============================] - 0s 0us/step


In [14]:
with strategy.scope():

  # define a sequential model
  model = Sequential([
                      tf.keras.layers.InputLayer(input_shape = (28, 28, 1)),
                      tf.keras.layers.BatchNormalization(),
                      tf.keras.layers.Conv2D(filters = 20, kernel_size = (3,3), activation = "relu", padding = "same"),
                      tf.keras.layers.MaxPool2D(strides = 2),
                      tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", padding = "same"),
                      tf.keras.layers.MaxPool2D(strides = 2),
                      tf.keras.layers.Flatten(),
                      tf.keras.layers.Dense(32, activation = tf.keras.layers.LeakyReLU(alpha=0.01)),
                      tf.keras.layers.Dense(10, activation = "softmax")
                    ])
  model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001), loss= tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics=[tf.keras.metrics.CategoricalAccuracy(name = "cat_acc")])
  model.fit(x_train, y_train, epochs = 2, batch_size = 32)

Epoch 1/2
1875/1875 [==============================] - 25s 12ms/step - loss: 0.5581 - cat_acc: 0.8428
Epoch 2/2
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1623 - cat_acc: 0.9523


In [15]:
with strategy.scope():

  # define a new model that takes input and gives out CNN features
  model2 = Sequential([tf.keras.layers.InputLayer(input_shape = (28, 28, 1)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Conv2D(filters = 20, kernel_size = (3,3), activation = "relu", padding = "same"),
                    tf.keras.layers.MaxPool2D(strides = 2),
                    tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", padding = "same"),
                    tf.keras.layers.MaxPool2D(strides = 2),
                    tf.keras.layers.Flatten()
                    ])
  
  # set the model weights and biases the same as from the trained model
  model2.layers[1].set_weights(model2.layers[1].get_weights())
  model2.layers[3].set_weights(model2.layers[3].get_weights())

  # pass training data through CNN layers and get CNN features
  predictions = model2(x_train).numpy()

In [ ]:
# define a default Support vector classifier
svc = SVC()

# pass training data through CNN layers and get CNN features
svc.fit(predictions, y_train.argmax(axis = 1))

In [ ]:
# try on testing data

# pass testing data through CNN layers and get CNN features
with strategy.scope():
  predictions = model2(x_test).numpy()

# get predictions from svc
predictions = svc.predict(predictions)

# evaluate the performance
print(classification_report(predictions, y_test.argmax(axis = 1)))

# Question 3

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

HSV_array = []

def dense_optical_flow(method, video_path, params=[], to_gray=False):
    # Read the video and first frame
    cap = cv.VideoCapture(video_path)
    ret, old_frame = cap.read()

    # crate HSV & make Value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255
    # Preprocessing for exact method
    if to_gray:
        old_frame = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    while True:
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break

        # Preprocessing for exact method
        if to_gray:
            new_frame = cv.cvtColor(new_frame, cv.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        # Use Hue and Value to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

        # Convert HSV image into BGR for demo
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
        # plt.imshow(frame_copy)
        # plt.show()
        HSV_array.append(bgr)
        k = cv.waitKey(25) & 0xFF
        if k == 27:
            break

        # Update the previous frame
        old_frame = new_frame

In [ ]:
dense_optical_flow(cv.optflow.calcOpticalFlowSparseToDense, "my_final_video.mp4")

In [ ]:
nrows = 8
ncols = 8

fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (20, 20))

for i in range(nrows):
  for j in range(ncols):
    ax[i, j].imshow(HSV_array[i*ncols + j])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(HSV_array[0])
plt.show()

## Sparse Optical Flow

In [ ]:
o_f_array = []

def lucas_kanade_method(video_path):
    # Read the video 
    cap = cv.VideoCapture(video_path)

    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03),
    )

    # Create random colors
    color = np.random.randint(0, 255, (100, 3))

    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    while True:
      # Read new frame
      ret, frame = cap.read()
      if not ret:
          break
      frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

      # Calculate Optical Flow
      p1, st, err = cv.calcOpticalFlowPyrLK(
          old_gray, frame_gray, p0, None, **lk_params
      )
      # Select good points
      good_new = p1[st == 1]
      good_old = p0[st == 1]

      # Draw the tracks
      for i, (new, old) in enumerate(zip(good_new, good_old)):
          a, b = new.ravel()
          c, d = old.ravel()
          mask = cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
          frame = cv.circle(frame, (a, b), 5, color[i].tolist(), -1)

      # Display the demo
      img = cv.add(frame, mask)
      o_f_array.append(img)
      k = cv.waitKey(25) & 0xFF
      if k == 27:
          break

      # Update the previous frame and previous points
      old_gray = frame_gray.copy()
      p0 = good_new.reshape(-1, 1, 2)

In [ ]:
lucas_kanade_method("my_final_video.mp4")

In [ ]:
nrows = 4
ncols = 4

fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (20, 20))

for i in range(nrows):
  for j in range(ncols):
    ax[i, j].imshow(o_f_array[::-1][i*ncols + j])

plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(o_f_array[18])
plt.show()